<a href="https://colab.research.google.com/github/mmsamiei/final-ms-thesis-code/blob/master/phase1/hello_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nlp
!pip install transformers
!pip install faiss-cpu

     |████████████████████████████████| 1.7MB 7.7MB/s 
     |████████████████████████████████| 17.2MB 240kB/s 
     |████████████████████████████████| 245kB 46.8MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 778kB 8.2MB/s 
     |████████████████████████████████| 890kB 23.5MB/s 
     |████████████████████████████████| 3.0MB 36.9MB/s 
     |████████████████████████████████| 1.1MB 45.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=846581663c05c7d98e957cddc0688e064b0bc3e556b2dfd62f5cd767ca67aa36
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 7.2MB 7.4MB/s 


In [47]:
from transformers import AutoTokenizer, AutoModel
import torch
torch.set_grad_enabled(False)
ctx_encoder = AutoModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
ctx_tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

In [96]:
from nlp import load_dataset
ds = load_dataset('squad_v2', split='train[:2000]')
ds_with_embeddings = ds.map(lambda example: {'embeddings': ctx_encoder(**ctx_tokenizer(example["question"], return_tensors="pt"))[0][0][0].numpy()})

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/12c16b15f6ee66809d3ca81ae2eeb4a5c9268a6899888bebf6f042774af8138a. Subsequent calls will reuse this data.


In [97]:
ds_with_embeddings.add_faiss_index(column='embeddings')

Dataset(features: {'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}, 'context': Value(dtype='string', id=None), 'embeddings': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'id': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None)}, num_rows: 2000)

In [98]:
q_encoder = AutoModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
q_tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

In [105]:
question = "How many did Crazy in Love sell to become one of the greatest selling singles in history?"
question_embedding = q_encoder(**q_tokenizer(question, return_tensors="pt"))[0][0][0].numpy()
scores, retrieved_examples = ds_with_embeddings.get_nearest_examples('embeddings', question_embedding, k=10)

In [106]:
retrieved_examples['question']

['How many did Crazy in Love sell to become one of the greatest selling singles in history?',
 'How many copies did Crazy in Love sell?',
 'Which publication considers Crazy in Love to be one of the top 500 songs of all time?',
 "How many records did Beyoncé sell as part of Destiny's Child?",
 'How many records has Beyoncé sold throughout the world?',
 'What is the title of his first commercially successful work?',
 'How much did the second world tour make in dollars?',
 "When Beyonce was with Destiny's Child, how many albums did she manage to sell?",
 "The world's best selling celebrity perfume line belongs to whom?",
 'How many Grammy awards did Crazy in Love get?']

# Saving and Loading


In [107]:
ds_with_embeddings.save_faiss_index('embeddings', 'my_index.faiss')

In [109]:
ds = load_dataset('squad_v2', split='train[:2000]')
ds.load_faiss_index('embeddings', 'my_index.faiss')